In [1]:
import numpy as np  
import pandas as pd
import re  
import scipy
import nltk

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
from pymystem3 import Mystem
from scipy.spatial import distance 

nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\v.kozlovskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
path = 'C:/Users/v.kozlovskiy/Desktop/DATAS/from_aug24/_ekklesiat_wc_and_other/to_git/'

In [3]:
m = Mystem() # lemmatizer for russian words

In [4]:
# open prepared file
text_df = pd.read_excel(path + 'text_for_analysis_rus.xlsx')
original = list(text_df['original'])
text = list(text_df['processed'])
text

['слово екклесиаст сын давидова царь в иерусалим',
 'суета суета сказать екклесиаст суета суета все суета',
 'что польза человек от весь труд его который трудиться он под солнце',
 'род проходить и род приходить а земля пребывать во веко',
 'восходить солнце и заходить солнце и спешить к место свой где оно восходить',
 'идти ветер к юг и переходить к север кружиться кружиться на ход свой и возвращаться ветер на круг свой',
 'весь река течь в море но море не переполняться к тот место откуда река течь они возвращаться чтобы опять течь',
 'весь вещь в труд не мочь человек пересказывать все не насыщаться око зрение не наполняться ухо слушание',
 'что быть то и быть и что делаться то и быть делаться и нет ничто новый под солнце',
 'бывать нечто о чем говорить смотреть вот это новое но это быть уже в век бывший прежде мы',
 'нет память о прежний да и о то что быть не оставаться память у тот который быть после',
 'я екклесиаст быть царь над израиль в иерусалим',
 'и предавать я сердце мой то 

In [5]:
def matrix_by_text(text):
    # preprocessing like before making wordcloud
    russian_stopwords = stopwords.words("russian")
    russian_stopwords.extend(['это', 'все', 'вс'])
    english_stopwords = stopwords.words("english")
    union = ' '.join(text)
    text_tokens = word_tokenize(union)
    text_tokens = [token.strip() for token in text_tokens if token not in russian_stopwords]
    
    # select unique tokens
    uniq_tokens = pd.Series(text_tokens).unique()

    # matrix for analysis in first dimension strings in text in second unique tokens
    matrix = np.zeros((len(text), len(uniq_tokens)))
    print('Number of strings in text:', len(text))
    print('Number of tokens in text:', len(text_tokens))
    print('Number of unique tokens in text:', len(uniq_tokens))
    print('Matrix shape:', matrix.shape)
    
    # cycle for fill the matrix (if corresponding token is in sentence 1, if no 0)
    for sentence in range(len(text)):
        single_sentence = re.split('[^а-я]', text[sentence])
        #print(single_sentence)
        for word in single_sentence:
            for i in range(len(uniq_tokens)):
                if word == uniq_tokens[i]:
                    #print(word, i)
                    matrix[sentence, i] = 1

    return matrix, uniq_tokens

In [6]:
# function for text filtration and lemmatization of inputed text
def filtration(text):
  text = text.lower()
  text = ''.join(m.lemmatize(text))
  text = re.sub(r'[^а-я\']', ' ', text)
  text = text.split()
  print('ok')
  return ' '.join(text)

In [7]:
# function for encoding of inputed text with unique tokens to compare it with matrix
def sentence_encoder(text, uniq_tokens):
    text = filtration(text)
    #print(text)
    if len(text) > len(uniq_tokens):
        text = text[:len(uniq_tokens)]
    encoded = np.zeros(len(uniq_tokens))
    for word in text.split():
        for i in range(len(uniq_tokens)):
            if word == uniq_tokens[i]:
                print(word, i)
                encoded[i] = 1
    return encoded, text

In [8]:
# function for counting cousine distance and define closest sentences from text and index or closest sentence
def nearest_phrase(encoded_input, matrix):

    index_list = []  # list for indexes
    distance_list = []  # list for cousine distances
    for i in range(matrix.shape[0]):  # cycle for compare encoded inputed phrase with phrases from file
         index_list.append(i)   
         distance_list.append(scipy.spatial.distance.cosine(encoded_input, matrix[i,:]))    # counting of cousine distance

    # table with results
    distance_table = pd.DataFrame([index_list, distance_list], index=['index', 'distance'])
    distance_table = distance_table.T
    #print(np.argmin(distance_table['distance']))
    needable_index = np.argmin(distance_table['distance'])
    distance_table = distance_table.sort_values(by='distance', ascending=True)
    #print(distance_table.head(5))
    
    return distance_table, needable_index

In [9]:
matrix_a, uniq_tokens = matrix_by_text(text)

Number of strings in text: 222
Number of tokens in text: 2214
Number of unique tokens in text: 797
Matrix shape: (222, 797)


In [10]:
input_text = input()
print('Inputed text:', input_text)

Inputed text: мудрость труд Солнце 12 труды человек


In [11]:
enc_input, phrase = sentence_encoder(input_text, uniq_tokens)
print('Processed inputed text:', phrase)

ok
мудрость 69
труд 11
солнце 14
труд 11
человек 9
Processed inputed text: мудрость труд солнце труд человек


In [12]:
distance_table, needable_index = nearest_phrase(enc_input, matrix_a)
print()
print('5 closest phrase from text:')
print()
for i in distance_table['index'][:5]:
    print(text_df['original'][i], round(distance_table['distance'][i],2))


5 closest phrase from text:

1:3 что пользы человеку от всех трудов его, которыми трудится он под солнцем? 0.43
2:18 и возненавидел я весь труд мой, которым трудился под солнцем, потому что должен оставить его человеку, который будет после меня. 0.5
2:22 ибо что будет иметь человек от всего труда своего и заботы сердца своего, что трудится он под солнцем? 0.5
9:13 вот еще какую мудрость видел я под солнцем, и она показалась мне важною: 0.55
7:11 хороша мудрость с наследством, и особенно для видящих солнце: 0.59
